---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [40]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import MinMaxScaler

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [3]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [4]:
def graph_identification():
    graph_id = []
    for P1_Graph in P1_Graphs:
        P1_Graph._node = P1_Graph.node
        P1_Graph._adj = P1_Graph.edge
        degrees = dict(P1_Graph.degree())
        degree_values = sorted(set(degrees.values()))
        degree_dist = [list(degrees.values()).count(i)/float(nx.number_of_nodes(P1_Graph)) for i in degree_values]
        clustering = nx.average_clustering(P1_Graph)
        shortest_path = nx.average_shortest_path_length(P1_Graph)

        # Check first 5 samples if it follows PA
        if degree_dist[:5] == sorted(degree_dist[:5], reverse=True):
            graph_id.append('PA')
        elif clustering > 0.1:
            graph_id.append('SW_L')
        else:
            graph_id.append('SW_H')        
        
    return graph_id

graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [5]:
G = nx.read_gpickle('email_prediction.txt')
G._node = G.node
G._adj = G.edge

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [26]:
def salary_predictions():
    df = pd.DataFrame(index=G.nodes())
    df['Department'] = pd.Series(dict(G.nodes(data=True))).map(lambda x: x['Department'])
    dummy = pd.get_dummies(df['Department'], prefix='Dept_')
    df = pd.concat([df, dummy], axis=1)
    df.drop('Department', axis=1, inplace=True)
    df['clustering'] = pd.Series(nx.clustering(G))
    df['degree_centrality'] = pd.Series(nx.degree_centrality(G))
    df['closeness_centrality'] = pd.Series(nx.closeness_centrality(G))
    df['page_rank'] = pd.Series(nx.pagerank(G, alpha=0.9))
    df['ManagementSalary'] = pd.Series(dict(G.nodes(data=True))).map(lambda x: x['ManagementSalary'])
    
    dfTarget = df[df['ManagementSalary'].isna()]
    df.dropna(inplace=True)
    
    X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,:-1] , df.iloc[:,-1], random_state=0)
    clf = LogisticRegression()
    param_grid = {'penalty': ['l1', 'l2'], 'C':[0.01, 0.1, 1, 10, 100]}
    
#     clf = SVC(kernel='rbf')
#     param_grid = {'gamma': [0.001, 0.01, 0.05, 0.1, 1, 10, 100]}
    
#     clf = RandomForestClassifier(n_estimators=50)
#     param_grid = {
#         'max_depth': [80, 90, 100],
#         'max_features': [2, 3],
#     }
    
    # Instantiate the grid search model
    grid_clf = GridSearchCV(estimator=clf, param_grid=param_grid, cv=3,
                               n_jobs=-1, verbose=2, scoring='roc_auc')
    
    grid_clf.fit(X_train, y_train)
    y_predict = grid_clf.predict(X_test)
    score = roc_auc_score(y_test, y_predict)
    
    X = dfTarget.iloc[:,:-1]
    y = grid_clf.predict_proba(X)[:, 1]
    
    
    return pd.Series(y, index=X.index)

salary_predictions()

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   7 out of  30 | elapsed:    0.0s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    0.5s finished
c:\users\patrick\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


1       1.103772e-03
2       7.024664e-01
5       9.941830e-01
8       7.783746e-02
14      1.917380e-01
18      5.108551e-03
27      3.670221e-01
30      6.431847e-01
31      2.509436e-01
34      1.658928e-01
37      5.496024e-02
40      1.745421e-01
45      8.202116e-02
54      5.490272e-01
55      6.277370e-01
60      8.240080e-02
62      9.856221e-01
65      3.769007e-01
77      1.268316e-02
79      4.190573e-02
97      1.022020e-03
101     3.168763e-04
103     2.975122e-01
108     4.063242e-02
113     8.796251e-01
122     2.543518e-04
141     2.074130e-01
142     9.046173e-01
144     2.940070e-02
145     4.873334e-01
            ...     
913     2.851831e-03
914     2.136308e-02
915     2.193518e-06
918     1.927757e-02
923     3.132997e-04
926     1.770045e-02
931     5.812328e-03
934     1.745010e-04
939     1.583280e-04
944     5.134981e-06
945     8.066683e-04
947     1.096247e-01
950     5.483093e-02
951     1.199688e-03
953     3.718036e-03
959     6.706964e-06
962     5.938

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [60]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [65]:
def new_connections_predictions():
    future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
    future_connections['common_neigh'] = [len(list(nx.common_neighbors(G, e[0], e[1]))) for e in future_connections.index]
    future_connections['Jaccard Coefficient'] = [i[2] for i in nx.jaccard_coefficient(G, future_connections.index)]
    future_connections['Preferential Attachment'] = [i[2] for i in nx.preferential_attachment(G, future_connections.index)]
    
    for node in G.nodes():
        G.node[node]['community'] = G.node[node]['Department']
    future_connections['cn_soundarajan_hopcroft'] = [i[2] for i in nx.cn_soundarajan_hopcroft(G, future_connections.index)]   
    future_connections['ra_soundarajan_hopcroft'] = [i[2] for i in nx.ra_index_soundarajan_hopcroft(G, future_connections.index)]
    
    dfTarget = future_connections[pd.isnull(future_connections['Future Connection'])]
    future_connections.dropna(inplace=True)
    
    X_train, X_test, y_train, y_test = train_test_split(future_connections.iloc[:,1:], 
                                                        future_connections.iloc[:,0], random_state=0)
    
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    clf = LogisticRegression()
    param_grid = {'penalty': ['l1', 'l2'], 'C':[0.01, 0.1, 1, 10, 100]}
    
#     clf = SVC(kernel='rbf')
#     param_grid = {'gamma': [0.001, 0.01, 0.05, 0.1, 1, 10, 100]}
    
#     clf = RandomForestClassifier(n_estimators=50)
#     param_grid = {
#         'max_depth': [80, 90, 100],
#         'max_features': [2, 3],
#     }
    
    # Instantiate the grid search model
    grid_clf = GridSearchCV(estimator=clf, param_grid=param_grid, cv=3,
                               n_jobs=-1, verbose=2, scoring='roc_auc')
    
    grid_clf.fit(X_train, y_train)
    y_predict = grid_clf.predict(X_test)
    score = roc_auc_score(y_test, y_predict)

    X = dfTarget.iloc[:,1:]
    y = grid_clf.predict_proba(X)[:, 1]
    
    return pd.Series(y, index=X.index)

new_connections_predictions()

c:\users\patrick\appdata\local\programs\python\python36\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:   47.9s finished
c:\users\patrick\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8106914396576065


(107, 348)    0.026977
(542, 751)    0.010537
(20, 426)     0.594991
(50, 989)     0.010594
(942, 986)    0.010654
(324, 857)    0.010586
(13, 710)     0.146572
(19, 271)     0.138724
(319, 878)    0.010613
(659, 707)    0.010543
(49, 843)     0.010631
(208, 893)    0.010557
(377, 469)    0.008352
(405, 999)    0.017819
(129, 740)    0.016790
(292, 618)    0.018172
(239, 689)    0.010617
(359, 373)    0.009226
(53, 523)     0.151304
(276, 984)    0.010628
(202, 997)    0.010640
(604, 619)    0.056535
(270, 911)    0.010616
(261, 481)    0.072234
(200, 450)    0.999991
(213, 634)    0.010520
(644, 735)    0.044497
(346, 553)    0.010379
(521, 738)    0.010108
(422, 953)    0.017622
                ...   
(672, 848)    0.010616
(28, 127)     0.935626
(202, 661)    0.010355
(54, 195)     0.999790
(295, 864)    0.010577
(814, 936)    0.010528
(839, 874)    0.010654
(139, 843)    0.010582
(461, 544)    0.009920
(68, 487)     0.009887
(622, 932)    0.010563
(504, 936)    0.016714
(479, 528) 